In [ ]:
import pandas as pd

timestamp = '20231122_133530'
#timestamp = '20231122_161757'
#timestamp = '20231116_151751'
timestamp = '20231129_133727'

# Directory path
#directory_path = f'/home/insectvr/src/build/20231116_StripeFixation_Data/RunData/{timestamp}/'
#directory_path = f'/Users/apaula/Nextcloud/locustVR/locustVR_data/20231122_RunData/{timestamp}/'
directory_path = f'/Users/apaula/Nextcloud/locustVR/locustVR_data/RunData_20231129/{timestamp}/'

# Full paths for each file
file_path1 = directory_path + f'{timestamp}_ChoiceAssay_VR1_.csv'
file_path2 = directory_path + f'{timestamp}_ChoiceAssay_VR2_.csv'
file_path3 = directory_path + f'{timestamp}_ChoiceAssay_VR3_.csv'
file_path4 = directory_path + f'{timestamp}_ChoiceAssay_VR4_.csv'

# Load the files into DataFrames
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)
df4 = pd.read_csv(file_path4)

# Parse dates in the "Current Time" column
df1['Current Time'] = pd.to_datetime(df1['Current Time'])
df2['Current Time'] = pd.to_datetime(df2['Current Time'])
df3['Current Time'] = pd.to_datetime(df3['Current Time'])
df4['Current Time'] = pd.to_datetime(df4['Current Time'])

# Display the first few rows of each DataFrame to confirm
print("DataFrame 1:")
print(df1.head())
print("\nDataFrame 2:")
print(df2.head())
print("\nDataFrame 3:")
print(df3.head())
print("\nDataFrame 4:")
print(df4.head())

In [ ]:
df1['source'] = 'df1'
df2['source'] = 'df2'
df3['source'] = 'df3'
df4['source'] = 'df4'

# Combine all DataFrames into one
df_combined = pd.concat([df1, df2, df3, df4])
df_combined['CurrentStep'].unique()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df1['source'] = 'df1'
df2['source'] = 'df2'
df3['source'] = 'df3'
df4['source'] = 'df4'

# Combine all DataFrames into one
df_combined = pd.concat([df1, df2, df3, df4])

# Function to plot polar histograms for each DataFrame side by side
def plot_combined_polar_histograms(df_combined, step):
    # Filter the DataFrame for the given scene value
    filtered_df = df_combined[df_combined['CurrentStep'] == step]
    
    # Get the unique sources
    sources = filtered_df['source'].unique()

    # Number of bins for the histogram
    num_bins = 360

    # Create a figure with subplots
    fig, axes = plt.subplots(1, len(sources), subplot_kw=dict(polar=True), figsize=(15, 6))

    for ax, source in zip(axes, sources):
        # Filter by source
        source_df = filtered_df[filtered_df['source'] == source]

        # Extract the GameObjectRotX values
        rot_x_values = source_df['GameObjectRotY']

        # Compute histogram
        counts, bin_edges = np.histogram(rot_x_values, bins=num_bins, range=(0, 2*np.pi))
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

        # Plot the histogram
        ax.bar(bin_centers, counts, width=bin_edges[1] - bin_edges[0], edgecolor='k')
        ax.set_title(f'{source} - CurrentStep {step}')
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)


    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()

# Example usage for a specific scene value
steps = df_combined['CurrentStep'].unique()
for scene in steps:
    plot_combined_polar_histograms(df_combined, scene)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots


# Assuming df_combined is your DataFrame
# Calculate the differences in position
df_combined['delta_x'] = df_combined['SensPosX'].diff()
df_combined['delta_y'] = df_combined['SensPosY'].diff()

# Calculate velocity (distance over time, here time is 1/60 second)
df_combined['velocity'] = np.sqrt(df_combined['delta_x']**2 + df_combined['delta_y']**2) * 60  # Multiplying by 60 as time interval is 1/60s
# Plotting histogram
plt.hist(df_combined[df_combined['velocity'] < 10]['velocity'].dropna(), bins=30)  # dropna to remove NaN values from the first row
plt.title('Velocity Histogram (Velocity < 10)')
plt.xlabel('Velocity (units per second)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import plotly.express as px

# Selecting the same subset of the DataFrame
subset_df = df_combined[df_combined['velocity'] <= 10]

# Create a histogram with Plotly for the subset
fig = px.histogram(subset_df, x='velocity', nbins=50000, title='Velocity Histogram (All Rows)')
fig.update_xaxes(title='Velocity (units per second)')
fig.update_yaxes(title='Count')
fig.show()


In [ ]:

import plotly.subplots as sp

# Create a figure with subplots
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=('df1', 'df2', 'df3', 'df4'))

# List of DataFrames
dfs = [df1, df2, df3, df4]

# Iterate over the DataFrames and create subplots
for i, df in enumerate(dfs):
    # Calculate the differences in position
    df['delta_x'] = df['SensPosX'].diff()
    df['delta_y'] = df['SensPosY'].diff()

    # Calculate velocity (distance over time, here time is 1/60 second)
    df['velocity'] = np.sqrt(df['delta_x']**2 + df['delta_y']**2) * 60  # Multiplying by 60 as time interval is 1/60s

    subset_df = df[df['velocity'] <= 10]
    # Applying a rolling window of 20 frames and calculating the mean velocity
    subset_df['rolling_velocity'] = subset_df['velocity'].rolling(window=20).mean()

    # Filtering out rolling velocities greater than 10 cm/s
    filtered_df = subset_df[subset_df['rolling_velocity'] <= 10]

    # Create a histogram subplot for the filtered, rolling average data
    fig.add_trace(px.histogram(filtered_df, x='rolling_velocity', nbins=150).data[0], row=(i // 2) + 1, col=(i % 2) + 1)

# Update subplot titles and axis labels
fig.update_layout(title='Rolling Average Velocity Histogram (20 Frames, <= 10 cm/s)', showlegend=False)
fig.update_xaxes(title_text='Rolling Average Velocity (cm per second)', row=1, col=1)
fig.update_xaxes(title_text='Rolling Average Velocity (cm per second)', row=1, col=2)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Count', row=2, col=1)

# Show the figure
fig.show()


In [ ]:
dfs = [df1, df2, df3, df4]

# Create a figure with subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('df1', 'df2', 'df3', 'df4'))

# Iterate over the DataFrames and create subplots
for i, df in enumerate(dfs):
    # Calculate the differences in position
    df['delta_x'] = df['SensPosX'].diff()
    df['delta_y'] = df['SensPosY'].diff()

    # Calculate velocity (distance over time, here time is 1/60 second)
    df['velocity'] = np.sqrt(df['delta_x']**2 + df['delta_y']**2) * 60  # Multiplying by 60 as time interval is 1/60s
    df['velocity'] = df['velocity']*0.45
    # Create a line plot for velocity against time
    fig.add_trace(go.Scatter(x=df['Current Time'], y=df['velocity'], mode='lines', name=f'Velocity df{i+1}'),
                  row=(i // 2) + 1, col=(i % 2) + 1)

# Update subplot titles and axis labels
fig.update_layout(title='Velocity Across Time for Each DataFrame', showlegend=False)
fig.update_xaxes(title_text='Time', row=1, col=1)
fig.update_xaxes(title_text='Time', row=1, col=2)
fig.update_yaxes(title_text='Velocity (cm per second)', row=1, col=1)
fig.update_yaxes(title_text='Velocity (cm per second)', row=2, col=1)



In [ ]:
dfs = [df1, df2, df3, df4]

# Create a figure with subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('df1', 'df2', 'df3', 'df4'))

# Iterate over the DataFrames and create subplots
for i, df in enumerate(dfs):
    # Calculate the differences in position
    df['delta_x'] = df['GameObjectPosX'].diff()
    df['delta_y'] = df['GameObjectPosZ'].diff()

    # Calculate velocity (distance over time, here time is 1/60 second)
    df['velocity'] = np.sqrt(df['delta_x']**2 + df['delta_y']**2) * 60  # Multiplying by 60 as time interval is 1/60s

    # Create a line plot for velocity against time
    fig.add_trace(go.Scatter(x=df['Current Time'], y=df['velocity'], mode='lines', name=f'Velocity df{i+1}'),
                  row=(i // 2) + 1, col=(i % 2) + 1)

# Update subplot titles and axis labels
fig.update_layout(title='Velocity Across Time for Each DataFrame', showlegend=False)
fig.update_xaxes(title_text='Time', row=1, col=1)
fig.update_xaxes(title_text='Time', row=1, col=2)
fig.update_yaxes(title_text='Velocity (cm per second)', row=1, col=1)
fig.update_yaxes(title_text='Velocity (cm per second)', row=2, col=1)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df1['source'] = 'df1'
df2['source'] = 'df2'
df3['source'] = 'df3'
df4['source'] = 'df4'

# Combine all DataFrames into one
df_combined = pd.concat([df1, df2, df3, df4])
df_combined['velocity_cm'] = df_combined['velocity']*0.45


# Function to plot polar histograms for each DataFrame side by side
def plot_combined_polar_histograms(df_combined, step):
    # Filter the DataFrame for the given scene value
    filtered_df = df_combined[(df_combined['CurrentStep'] == step) & (df_combined['velocity_cm'] >=0.2)] #& (df_combined['Current Time'] <= pd.Timestamp("2023-11-22 13:55:52.897"))]
    
    # Get the unique sources
    sources = filtered_df['source'].unique()

    # Number of bins for the histogram
    num_bins = 36

    # Create a figure with subplots
    fig, axes = plt.subplots(1, len(sources), subplot_kw=dict(polar=True), figsize=(15, 6))

    for ax, source in zip(axes, sources):
        # Filter by source
        source_df = filtered_df[filtered_df['source'] == source]

        # Extract the GameObjectRotY and SensRotY values
        game_object_rot_y = source_df['GameObjectRotY']
        sens_rot_y = source_df['SensRotY']

        # Compute histograms
        game_object_counts, game_object_bin_edges = np.histogram(game_object_rot_y, bins=num_bins, range=(0, 2*np.pi))
        sens_counts, sens_bin_edges = np.histogram(sens_rot_y, bins=num_bins, range=(0, 2*np.pi))

        # Compute bin centers
        game_object_bin_centers = (game_object_bin_edges[:-1] + game_object_bin_edges[1:]) / 2
        sens_bin_centers = (sens_bin_edges[:-1] + sens_bin_edges[1:]) / 2

        # Plot the histograms
        ax.bar(game_object_bin_centers, game_object_counts, width=game_object_bin_edges[1] - game_object_bin_edges[0], edgecolor='k', alpha=0.5, label='GameObjectRotY')
        #ax.bar(sens_bin_centers, sens_counts, width=sens_bin_edges[1] - sens_bin_edges[0], edgecolor='r', alpha=0.5, label='SensRotY', color='r')

        ax.set_title(f'{source} - CurrentStep {step}')
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)

    # Add legend and adjust layout
    axes[0].legend(loc='upper right')
    plt.tight_layout()
    plt.show()

# Example usage for a specific step value
steps = df_combined['CurrentStep'].unique()
for step in steps:
    plot_combined_polar_histograms(df_combined, step)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df1['source'] = 'df1'
df2['source'] = 'df2'
df3['source'] = 'df3'
df4['source'] = 'df4'

# Combine all DataFrames into one
df_combined = pd.concat([df1, df2, df3, df4])
df_combined['velocity_cm'] = df_combined['velocity']*0.45


# Function to plot histograms for each DataFrame side by side
def plot_combined_histograms(df_combined, step):
    # Filter the DataFrame for the given step value
    filtered_df = df_combined[(df_combined['CurrentStep'] == step) & (df_combined['velocity_cm'] >=0.2)]

    
    # Get the unique sources
    sources = filtered_df['source'].unique()

    # Number of bins for the histogram
    num_bins = 50

    # Create a figure with subplots
    fig, axes = plt.subplots(1, len(sources), figsize=(15, 6))

    for ax, source in zip(axes, sources):
        # Filter by source
        source_df = filtered_df[filtered_df['source'] == source]

        # Extract the GameObjectRotY and SensRotY values
        game_object_rot_y = source_df['GameObjectRotY']
        sens_rot_y = source_df['SensRotY']

        # Plot the histograms
        ax.hist(game_object_rot_y, bins=num_bins, alpha=0.5, label='GameObjectRotY')
        #ax.hist(sens_rot_y, bins=num_bins, alpha=0.5, label='SensRotY', color='r')

        ax.set_title(f'{source} - CurrentStep {step}')
        ax.set_xlabel('Rotation Y')
        ax.set_ylabel('Frequency')

    # Add legend and adjust layout
    axes[0].legend(loc='upper right')
    plt.tight_layout()
    plt.show()

# Example usage for a specific step value
steps = df_combined['CurrentStep'].unique()
for step in steps:
    plot_combined_histograms(df_combined, step)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming dfs = [df1, df2, df3, df4] contains your DataFrames

# Create a figure with subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('df1', 'df2', 'df3', 'df4'))

# Iterate over the DataFrames and create subplots
for i, df in enumerate(dfs):
    # Create a line plot for GameObjectRotY against time
    fig.add_trace(go.Scatter(x=df['Current Time'], y=df['GameObjectRotY'], mode='lines', name=f'GameObjectRotY df{i+1}'),
                  row=(i // 2) + 1, col=(i % 2) + 1)

    # Create a line plot for SensRotY against time
    fig.add_trace(go.Scatter(x=df['Current Time'], y=df['SensRotY'], mode='lines', name=f'SensRotY df{i+1}'),
                  row=(i // 2) + 1, col=(i % 2) + 1)

# Update subplot titles and axis labels
fig.update_layout(title='Rotation Y Across Time for Each DataFrame', showlegend=True)
fig.update_xaxes(title_text='Time', row=1, col=1)
fig.update_xaxes(title_text='Time', row=1, col=2)
fig.update_yaxes(title_text='Rotation Y (degrees)', row=1, col=1)
fig.update_yaxes(title_text='Rotation Y (degrees)', row=2, col=1)

# Show the figure
fig.show()
